In [ ]:
#####
#####  RUN NOTES
#####



#####
#####  PURPOSE
#####



import datetime 
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
#import
import pandas as pd
import numpy as np
import hddm
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pymc
import kabuki
sns.set(style="white")
%matplotlib inline
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
# estimate convergence
from kabuki.analyze import gelman_rubin

In [ ]:
data = hddm.load_csv('rlddm_data.csv')
#check structure
data.head()

In [ ]:
#run the model by calling hddm.HDDMrl (instead of hddm.HDDM for normal HDDM)
m = hddm.HDDMrl(data)
#set sample and burn-in
m.sample(500,burn=100,dbname='traces.db',db='pickle')
#print stats to get an overview of posterior distribution of estimated parameters
m.print_stats()

# Regular RL without RT

In [ ]:
#run the model by calling hddm.Hrl (instead of hddm.HDDM for normal model and hddm.HDDMrl for rlddm-model)
m_rl = hddm.Hrl(data)
#set sample and burn-in
m_rl.sample(1500,burn=500,dbname='traces.db',db='pickle')
#print stats to get an overview of posterior distribution of estimated parameters
m_rl.print_stats()

In [ ]:
# estimate convergence
models = []
for i in range(3):
    m = hddm.Hrl(data=data)
    m.sample(500, burn=200,dbname='traces.db',db='pickle')
    models.append(m)
#get max gelman-statistic value. shouldn't be higher than 1.1
np.max(list(gelman_rubin(models).values()))

In [ ]:
gelman_rubin(models)

In [ ]:
# Combine the models we ran to test for convergence.
m_rl = kabuki.utils.concat_models(models)

In [ ]:
alpha, v = m_rl.nodes_db.node[['alpha','v']]
samples = {'alpha':alpha.trace(),'v':v.trace()}
samp = pd.DataFrame(data=samples)

def corrfunc(x, y, **kws):
    r, _ = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.1, .9), xycoords=ax.transAxes)

g = sns.PairGrid(samp, palette=["red"])
g.map_upper(plt.scatter, s=10)
g.map_diag(sns.distplot, kde=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")

g.map_lower(corrfunc)

# Posterior predictive checks

In [ ]:
#create empty dataframe to store simulated data
sim_data = pd.DataFrame()
#create a column samp to be used to identify the simulated data sets
data['samp'] = 0
#load traces
traces = m_rl.get_traces()
#decide how many times to repeat simulation process. repeating this multiple times is generally recommended as it better captures the uncertainty in the posterior distribution, but will also take some time
for i in tqdm(range(1,51)):
    #randomly select a row in the traces to use for extracting parameter values
    sample = np.random.randint(0,traces.shape[0]-1)
    #loop through all subjects in observed data
    for s in data.subj_idx.unique():
        #get number of trials for each condition.
        size0 = len(data[(data['subj_idx']==s) & (data['split_by']==0)].trial.unique())
        size1 = len(data[(data['subj_idx']==s) & (data['split_by']==1)].trial.unique())
        size2 = len(data[(data['subj_idx']==s) & (data['split_by']==2)].trial.unique())
        #set parameter values for simulation
        scaler = traces.loc[sample,'v_subj.'+str(s)]
        alphaInv = traces.loc[sample,'alpha_subj.'+str(s)]
        #take inverse logit of estimated alpha
        alpha = np.exp(alphaInv)/(1+np.exp(alphaInv))
        #simulate data for each condition changing only values of size, p_upper, p_lower and split_by between conditions.
        sim_data0 = hddm.generate.gen_rand_rl_data(scaler=scaler,alpha=alpha,size=size0,p_upper=0.8,p_lower=0.2,split_by=0)
        sim_data1 = hddm.generate.gen_rand_rl_data(scaler=scaler,alpha=alpha,size=size1,p_upper=0.7,p_lower=0.3,split_by=1)
        sim_data2 = hddm.generate.gen_rand_rl_data(scaler=scaler,alpha=alpha,size=size2,p_upper=0.6,p_lower=0.4,split_by=2)
        #append the conditions
        sim_data0 = sim_data0.append([sim_data1,sim_data2],ignore_index=True)
        #assign subj_idx
        sim_data0['subj_idx'] = s
        #identify that these are simulated data
        sim_data0['type'] = 'simulated'
        #identify the simulated data
        sim_data0['samp'] = i
        #append data from each subject
        sim_data = sim_data.append(sim_data0,ignore_index=True)
#combine observed and simulated data
ppc_rl_data = data[['subj_idx','response','split_by','trial','feedback','samp']].copy()
ppc_rl_data['type'] = 'observed'
ppc_rl_sdata = sim_data[['subj_idx','response','split_by','trial','feedback','type','samp']].copy()
ppc_rl_data = ppc_rl_data.append(ppc_rl_sdata)

In [ ]:
#create empty dataframe to store simulated data
sim_data = pd.DataFrame()
#create a column samp to be used to identify the simulated data sets
data['samp'] = 0
#load traces
traces = m_rl.get_traces()
#decide how many times to repeat simulation process. repeating this multiple times is generally recommended as it better captures the uncertainty in the posterior distribution, but will also take some time
for i in tqdm(range(1,11)):
    #randomly select a row in the traces to use for extracting parameter values
    sample = np.random.randint(0,traces.shape[0]-1)
    #loop through all subjects in observed data
    for s in data.subj_idx.unique():
        #get number of trials for each condition.
        size0 = len(data[(data['subj_idx']==s) & (data['split_by']==0)].trial.unique())
        size1 = len(data[(data['subj_idx']==s) & (data['split_by']==1)].trial.unique())
        size2 = len(data[(data['subj_idx']==s) & (data['split_by']==2)].trial.unique())
        #set parameter values for simulation
        scaler = traces.loc[sample,'v_subj.'+str(s)]
        alphaInv = traces.loc[sample,'alpha_subj.'+str(s)]
        #take inverse logit of estimated alpha
        alpha = np.exp(alphaInv)/(1+np.exp(alphaInv))
        #simulate data for each condition changing only values of size, p_upper, p_lower and split_by between conditions.
        sim_data0 = hddm.generate.gen_rand_rl_data(scaler=scaler,alpha=alpha,size=size0,p_upper=0.8,p_lower=0.2,split_by=0)
        sim_data1 = hddm.generate.gen_rand_rl_data(scaler=scaler,alpha=alpha,size=size1,p_upper=0.7,p_lower=0.3,split_by=1)
        sim_data2 = hddm.generate.gen_rand_rl_data(scaler=scaler,alpha=alpha,size=size2,p_upper=0.6,p_lower=0.4,split_by=2)
        #append the conditions
        sim_data0 = sim_data0.append([sim_data1,sim_data2],ignore_index=True)
        #assign subj_idx
        sim_data0['subj_idx'] = s
        #identify that these are simulated data
        sim_data0['type'] = 'simulated'
        #identify the simulated data
        sim_data0['samp'] = i
        #append data from each subject
        sim_data = sim_data.append(sim_data0,ignore_index=True)
#combine observed and simulated data
ppc_rl_data = data[['subj_idx','response','split_by','trial','feedback','samp']].copy()
ppc_rl_data['type'] = 'observed'
ppc_rl_sdata = sim_data[['subj_idx','response','split_by','trial','feedback','type','samp']].copy()
ppc_rl_data = ppc_rl_data.append(ppc_rl_sdata)

In [ ]:
#for practical reasons we only look at the first 40 trials for each subject in a given condition
plot_ppc_rl_data = ppc_rl_data[ppc_rl_data.trial<41].copy()

In [ ]:
#bin trials to for smoother estimate of response proportion across learning
plot_ppc_rl_data['bin_trial'] = pd.cut(plot_ppc_rl_data.trial,11,labels=np.linspace(0, 10,11)).astype('int64')
#calculate means for each sample
sums = plot_ppc_rl_data.groupby(['bin_trial','split_by','samp','type']).mean().reset_index()
#calculate the overall mean response across samples
ppc_rl_sim = sums.groupby(['bin_trial','split_by','type']).mean().reset_index()
#initiate columns that will have the upper and lower bound of the hpd
ppc_rl_sim['upper_hpd'] = 0
ppc_rl_sim['lower_hpd'] = 0
for i in range(0,ppc_rl_sim.shape[0]):
    #calculate the hpd/hdi of the predicted mean responses across bin_trials
    hdi = pymc.utils.hpd(sums.response[(sums['bin_trial']==ppc_rl_sim.bin_trial[i]) & (sums['split_by']==ppc_rl_sim.split_by[i]) & (sums['type']==ppc_rl_sim.type[i])],alpha=0.1)
    ppc_rl_sim.loc[i,'upper_hpd'] = hdi[1]
    ppc_rl_sim.loc[i,'lower_hpd'] = hdi[0]
#calculate error term as the distance from upper bound to mean
ppc_rl_sim['up_err'] = ppc_rl_sim['upper_hpd']-ppc_rl_sim['response']
ppc_rl_sim['low_err'] = ppc_rl_sim['response']-ppc_rl_sim['lower_hpd']
ppc_rl_sim['model'] = 'RL'

In [ ]:
#plotting evolution of choice proportion for best option across learning for observed and simulated data. Compared for RL and RLDDM models, both with single learnign rate.
fig, axs = plt.subplots(figsize=(15, 5),nrows=1, ncols=3, sharex=True,sharey=True)
for i in range(0,3):
    ax = axs[i]
    d_single = ppc_rl_sim[(ppc_rl_sim.split_by==i) & (ppc_rl_sim.type=='simulated')]
    #slightly move bin_trial to avoid overlap in errorbars
    d_single['bin_trial'] += 0.2
    ax.errorbar(d_single.bin_trial, d_single.response, yerr=[d_single.low_err,d_single.up_err], label='simulated_RLDDM',color='orange')
    ax = axs[i]
    d_rl = ppc_rl_sim[(ppc_rl_sim.split_by==i) & (ppc_rl_sim.type=='simulated')]
    ax.errorbar(d_rl.bin_trial, d_rl.response, yerr=[d_rl.low_err,d_rl.up_err], label='simulated_RL',color='green')
    ax = axs[i]
    # d = ppc_rl_sim[(ppc_dual_sim.split_by==i) & (ppc_dual_sim.type=='observed')]
    # ax.plot(d.bin_trial, d.response,linewidth=3,label='observed')
    ax.set_title('split_by = %i' %i,fontsize=20)
    ax.set_ylabel('mean response')
    ax.set_xlabel('trial')
plt.xlim(-0.5,10.5)
plt.legend()

In [ ]:
# Specify number of samples and burnins
nsamples = 100
nburn = 50

m = hddm.HDDMnnRL(
    data, 
    model='angle', 
    rl_rule='RWupdate', 
    non_centered=True, 
    include=['z', 'theta', 'rl_alpha'], 
    p_outlier = 0.0
)
m.sample(nsamples, burn=nburn, dbname='traces.db', db='pickle')

In [ ]:
model_ssm = 'angle'
model_rl = 'RWupdate'

config_ssm = hddm.model_config.model_config[model_ssm]
config_rl = hddm.model_config_rl.model_config_rl[model_rl]

In [ ]:
m.save('rlssm_model')


In [ ]:
m.plot_posteriors()


In [ ]:
import hddm
import pickle
import pandas as pd

In [ ]:
# Load the trace
with open('./traces.db', 'rb') as handle:
    tracefile = pickle.load(handle)

In [ ]:
# Re-format traces as a dataframe
traces = hddm.utils.get_traces_rlssm(tracefile)

In [ ]:
model_ssm = 'angle'
model_rl = 'RWupdate'

config_ssm = hddm.model_config.model_config[model_ssm]
config_rl = hddm.model_config_rl.model_config_rl[model_rl]

In [ ]:
hddm.plotting.plot_posterior_pairs_rlssm(tracefile, config_ssm['params'] + config_rl['params'])


In [ ]:
num_posterior_samples = 3
p_lower = {0: 0.15, 1:0.30, 2:0.45}
p_upper = {0: 0.85, 1:0.70, 2:0.55}
ppc_sdata = hddm.plotting.gen_ppc_rlssm(model_ssm, config_ssm, model_rl, config_rl, data, traces, num_posterior_samples, p_lower, p_upper, save_data=True, save_name='ppc_data')


In [ ]:
_ = hddm.plotting.plot_ppc_choice_rlssm(data, ppc_sdata, 40, 10)


In [ ]:
_ = hddm.plotting.plot_ppc_rt_rlssm(data, ppc_sdata, 40, 0.06)
